In [381]:
import lib.initShared  # custom library
import os
cwd = os.getcwd()
lib.initShared.initializeSharedLib(cwd)
import pandas as pd

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('HomeCredit').getOrCreate()

In [382]:
data=spark.read.parquet(cwd+'/data/dataset')

In [383]:

type_dict={}
dict_counter={}
stat_dict_num={}
stat_dict_cat={}

def get_null_values():
    import pandas as pd
    profile=pd.DataFrame(columns=['column_name','null_count'])
    column_names=data.columns
    global dict_counter
    null_count_list=[]
    for col in column_names:
        null_count=data.filter(data[col].isNull()).count()
        null_count_list.append(null_count)
        dict_counter[col]=null_count
    profile['column_name']=column_names
    profile['null_count']=null_count_list
def get_type():
    global type_dict
    for col,ty in data.dtypes:
        type_dict[str(col)]=str(ty)
    dataF=pd.DataFrame()
    dataF['name']=type_dict.keys()
    dataF['type']=type_dict.values()
def get_stats(column_list):
    for col in column_list:

        stat_dict=data.select(col).summary().rdd.collectAsMap()
        stat_dict_num[col]=stat_dict
def cate_stats(columns):
    for col in columns:
        distinct=data.select(col).distinct().count()
        stat_dict_cat[col]=distinct
get_null_values()
get_type()
numeric_col=[i[0] for i in data.dtypes if i[1] != 'string']
get_stats( column_list=numeric_col)
cat_col=[i[0] for i in data.dtypes if i[1] == 'string']
cate_stats(cat_col)
profiler=pd.DataFrame()
#print(type_dict)
for i in numeric_col:
    this_dict={'Column':[i],'type':[type_dict[i]],'Null_count':[dict_counter[i]],'%Null':["{0:.2f}".format((dict_counter[i]/data.count())*100)]}
    for j in stat_dict_num[i].items():
        this_dict[j[0]]=["{0:.2f}".format(float(j[1]))]
    _temp=pd.DataFrame.from_dict(this_dict)
    profiler=pd.concat([profiler,_temp],axis=0)
profiler=profiler.set_index(['Column'])

cate_profiler=pd.DataFrame()
def nonasciitoascii(unicodestring):
    global count
    if unicodestring.isascii():
        count+=1
    return unicodestring.isascii()
convertedudf = udf(nonasciitoascii)
for i in cat_col:
    count=0
    this_dict={'Column':[i],'type':[type_dict[i]],'Null_count':[dict_counter[i]],'%Null':[(dict_counter[i]/data.count())*100]}
    distict_count=data.select(i).distinct().count()
    this_dict['categories']=distict_count
    
    count_dict=data.cube(i).count()
    count_dict=count_dict.rdd.collectAsMap()
    if dict_counter[i]==0:
        del(count_dict[None])
    sorted_by_value = sorted(count_dict.items(), key=lambda kv: kv[1])
    string=''
    
    
    converted = data.select('NAME_CONTRACT_TYPE').withColumn('con',convertedudf(data.NAME_CONTRACT_TYPE))
    this_dict['Non_ASCII']=count
    for idx,st in enumerate(sorted_by_value[-5:]):
        this_dict['Feature_'+str(idx)]=str(st[0])+':'+str(st[1])
    #printtthis_dict_dict_dict_dict_dicts_dicts_dictis_dict)
    
    _temp=pd.DataFrame.from_dict(this_dict)
    cate_profiler=pd.concat([cate_profiler,_temp],axis=0,ignore_index=True,sort=False)
cate_profiler=cate_profiler.set_index('Column')

In [384]:
cate_profiler

,type,Null_count,%Null,categories,Non_ASCII,Feature_0,Feature_1,Feature_2,Feature_3,Feature_4
Column,,,,,,,,,,
NAME_CONTRACT_TYPE,string,0,0.000000,2,0,Revolving loans:29279,Cash loans:278232,NaN,NaN,NaN
CODE_GENDER,string,0,0.000000,3,0,XNA:4,M:105059,F:202448,NaN,NaN
FLAG_OWN_CAR,string,0,0.000000,2,0,Y:104587,N:202924,NaN,NaN,NaN
FLAG_OWN_REALTY,string,0,0.000000,2,0,N:94199,Y:213312,NaN,NaN,NaN
NAME_TYPE_SUITE,string,1292,0.420148,8,0,Other_B:1770,Children:3267,"Spouse, partner:11370",Family:40149,Unaccompanied:248526
NAME_INCOME_TYPE,string,0,0.000000,8,0,Unemployed:22,State servant:21703,Pensioner:55362,Commercial associate:71617,Working:158774
NAME_EDUCATION_TYPE,string,0,0.000000,5,0,Academic degree:164,Lower secondary:3816,Incomplete higher:10277,Higher education:74863,Secondary / secondary special:218391
NAME_FAMILY_STATUS,string,0,0.000000,6,0,Widow:16088,Separated:19770,Civil marriage:29775,Single / not married:45444,Married:196432
NAME_HOUSING_TYPE,string,0,0.000000,6,0,Office apartment:2617,Rented apartment:4881,Municipal apartment:11183,With parents:14840,House / apartment:272868


In [385]:
profiler

,type,Null_count,%Null,count,mean,stddev,min,25%,50%,75%,max
Column,,,,,,,,,,,
SK_ID_CURR,int,0,0.00,307511.00,278180.52,102790.18,100002.00,189126.00,278178.00,367120.00,456255.00
TARGET,int,0,0.00,307511.00,0.08,0.27,0.00,0.00,0.00,0.00,1.00
CNT_CHILDREN,int,0,0.00,307511.00,0.42,0.72,0.00,0.00,0.00,1.00,19.00
AMT_INCOME_TOTAL,double,0,0.00,307511.00,168797.92,237123.15,25650.00,112500.00,147150.00,202500.00,117000000.00
AMT_CREDIT,double,0,0.00,307511.00,599026.00,402490.78,45000.00,270000.00,513531.00,808650.00,4050000.00
AMT_ANNUITY,double,12,0.00,307499.00,27108.57,14493.74,1615.50,16524.00,24903.00,34596.00,258025.50
AMT_GOODS_PRICE,double,278,0.09,307233.00,538396.21,369446.46,40500.00,238500.00,450000.00,679500.00,4050000.00
REGION_POPULATION_RELATIVE,double,0,0.00,307511.00,0.02,0.01,0.00,0.01,0.02,0.03,0.07
DAYS_BIRTH,int,0,0.00,307511.00,-16037.00,4363.99,-25229.00,-19682.00,-15750.00,-12413.00,-7489.00
